In [ ]:
# ASTR400B Research Assignment3
# Prof: G. Besla
# TA: R. Li
#Student: Yuxuan Chen

In [1]:
# import modules
import numpy as np
import astropy.units as u
from astropy.constants import G

# import plotting modules
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

# my modules
from ReadFile import Read
# Step 1: modify CenterOfMass so that COM_P now takes a parameter specifying 
# by how much to decrease RMAX instead of a factor of 2
from CenterOfMass2 import CenterOfMass


In [2]:
def MassEnclosed_M33(filename1, ptype, Rg, COM1_position):
    # Store the the position of COM for M33
    time, total, data = Read(filename1)
    index = np.where(data['type'] == ptype)
    x = data['x'][index]
    y = data['y'][index]
    z = data['z'][index]
    m = data['m'][index]
    # Read the file from VLowRes
    # Store positions and mass of particles of given ptype 
    xG = x - COM1_position[0]
    yG = y - COM1_position[1]
    zG = z - COM1_position[2]
    # Compute the magnitude of radius to COM
    rG = np.sqrt(xG**2 + yG**2 + zG**2)
    # store mass of particles of a given ptype
    mG = m
    indexR = np.where(rG < Rg)
    return np.sum(mG[indexR])
    
def MassEnclosed_M31(filename2,ptype, Rg, COM1_position):
    time, total, data = Read(filename2)
    index = np.where(data['type'] == ptype)
    x = data['x'][index]
    y = data['y'][index]
    z = data['z'][index]
    m = data['m'][index]
    # Store the the position of M31 to COM of M33
    # Read the file from VLowRes
    #Store positions and mass of particles of given ptype
    xG = x - COM1_position[0]
    yG = y - COM1_position[1]
    zG = z - COM1_position[2]
    # Compute the magnitude of radius to COM
    rG = np.sqrt(xG**2 + yG**2 + zG**2)
    # store mass of particles of a given ptype
    mG = m
    indexR = np.where(rG < Rg)
    return np.sum(mG[indexR])   
    
        
    
    
    
    


In [3]:
def Jacobi_R_M_profile(galaxy1,galaxy2,start,end,n):
    """function that loops over all the desired snapshots to Jacobi Radius,Jacobi Satellite Mass, Jacobi host mass
           galaxy1  is the the name of the satellite galaxy
           galaxy2  is the name of nearby galaxy
           start   initial SnapNumber  e.g. 0
           end     final SnapNumber  e.g. 100
           n       integer indicating the frequency with which the snapshots are read in;  n should not be 0
    returns:  file with COM pos/vel of a galaxy at snapshots over the range (start, end+n, n) 
                     columns t, r, Ms, Mh, rj  for n snapshots
    """
    
    # compose the filename for output
    #fileout = "Orbit_" + galaxy + ".txt"
    fileout = 'Orbit_%s.txt'%(galaxy1+galaxy2)   # filename to store output

    # Because my goal is really accurately, for M31 delta, VolDec = 0.1, 2.0
    # for M33 that is stripped more, use different values, delta, VolDec = 0.1, 4.0
    #M33 is galaxy1, M31 is galaxy2
    # generate the snapshot id sequence and check if the input is eligible
    snap_ids = np.arange(start, end+n, n)
    if snap_ids.size == 0:
        raise ValueError("Cannot build a sequence using the input, start = ", 
                         start, "end = ", end, "n = ", n)
    
    # initialize the array for orbital info: t, r,Ms,Mh,rj
    orbit = np.zeros([snap_ids.size, 5])
    R_jb = np.zeros(snap_ids.size)
    
    # This loop is going to write and calculate the data we need to new file
    for i, snap_id in enumerate(snap_ids): # loop over files
        
        # compose the data filename (be careful about the folder)
        filename1 = "./VLowRes/" + galaxy1 +  "_{:03d}".format(snap_id) + ".txt"
        filename2 = "./VLowRes/" + galaxy2 +  "_{:03d}".format(snap_id) + ".txt"
        
        # Initialize CenterOfMass for 2 galaxies, using disk particles
        COM1 = CenterOfMass(filename1, 2)
        COM2 = CenterOfMass(filename2, 2)
        # Store the the position of COM for 2 galaxies, now COM_P required VolDec
        COM1_position = COM1.COM_P(0.1, 4.0).value
        COM2_position = COM2.COM_P(0.1, 2.0).value
        #print to check
        print(COM1_position,COM2_position)
        #now calculate the distance between COM of galaxy1 and COM of galaxy2
        R_g =np.sqrt((COM1_position[0] - COM2_position[0])**2 + (COM1_position[1] - COM2_position[1])**2 +\
                     (COM1_position[2] - COM2_position[2])**2 )
        
        M_host = MassEnclosed_M33(filename1, 1.0, R_g, COM1_position) + MassEnclosed_M33(filename1, 2.0, R_g, COM1_position)+\
        MassEnclosed_M31(filename2, 1.0, R_g, COM1_position) + MassEnclosed_M31(filename2, 2.0, R_g, COM1_position) +\
        MassEnclosed_M31(filename2, 3.0, R_g, COM1_position)
        if i == 0:
            M_jb = MassEnclosed_M33(filename1, 1.0, R_g, COM1_position) + MassEnclosed_M33(filename1, 2.0, R_g, COM1_position)
            R_jb[i] = R_g*(M_jb/(2*M_host))**(1/3)
            
        else:
            M_jb = MassEnclosed_M33(filename1, 1.0,R_jb[i-1], COM1_position) + MassEnclosed_M33(filename1, 2.0,R_jb[i-1], COM1_position)
            R_jb[i] = R_g*(M_jb/(2*M_host))**(1/3)
            
            
        print("host mass:",M_host, "satellite mass:",M_jb, "Jacobi Radius:",R_jb[i])
        # store the time, M31M33distance, M_host, M_satellite, Jacobi radius without units (.value) 
        #orbit[i] = COM1.time.value, *tuple(round(R_g,3), *tuple(round(M_jb,3), *tuple(M_host), *tuple(R_jb[i])
        
        # print snap_id to see the progress
        print("snap id:",snap_id)
        
    # write the data to a file
    #np.savetxt(fileout, orbit, fmt = "%11.3f"*5, comments='#',
    #           header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}"\
    #                  .format('t', 'kpc', '1e12Msun', '1e12Msun', 'kpc'))

In [4]:
# read in 800 snapshots in intervals of n=5
# Note: This might take a little while - test your code with a smaller number of snapshots first! 

print("Starting to Compute M33 Jabico radius profile")
Jacobi_R_M_profile("M33","M31",0,800,5)


Starting to Compute M33 Jabico radius profile
[-476.22  491.44 -412.41] [-377.77  611.47 -284.6 ]
host mass: 71.54284190399999 satellite mass: 15.754536720000003 Jacobi Radius: 96.37887934265062
snap id: 0
[-472.95  498.95 -402.04] [-372.42  606.23 -281.09]
host mass: 69.954199298 satellite mass: 12.83217714 Jacobi Radius: 85.85613559897904
snap id: 5
[-469.31  506.76 -391.42] [-367.22  600.88 -277.49]
host mass: 67.03790665 satellite mass: 12.1081187 Jacobi Radius: 80.58376807767776
snap id: 10
[-465.05  515.03 -380.3 ] [-360.96  594.79 -273.71]
host mass: 68.53052944800001 satellite mass: 12.052134800000001 Jacobi Radius: 75.14713795219228
snap id: 15
[-460.2   523.51 -368.47] [-356.48  589.85 -270.41]
host mass: 64.64039354999998 satellite mass: 11.62665716 Jacobi Radius: 70.51966884739346
snap id: 20
[-454.66  532.67 -355.78] [-351.28  584.14 -266.96]
host mass: 64.61261831200001 satellite mass: 11.49602806 Jacobi Radius: 65.03741221376627
snap id: 25
[-448.1   541.96 -342.36] [-34

[ 14.48 156.6  -19.65] [-75.74 256.19 -34.92]
host mass: 62.525639012 satellite mass: 6.184922080000001 Jacobi Radius: 49.64406074691887
snap id: 265
[  2.47 140.76 -24.24] [-66.47 239.15 -30.99]
host mass: 59.44571535400001 satellite mass: 6.196218860000001 Jacobi Radius: 44.94618717507765
snap id: 270
[-12.02 126.7  -29.65] [-58.44 219.15 -28.24]
host mass: 56.834792684 satellite mass: 5.793134780000001 Jacobi Radius: 38.358261936815715
snap id: 275
[-28.67 115.67 -35.7 ] [-53.56 197.83 -29.5 ]
host mass: 51.310226074000006 satellite mass: 5.502018500000001 Jacobi Radius: 32.455345912352165
snap id: 280
[-47.   108.42 -41.57] [-51.6  179.82 -35.19]
host mass: 43.922469566000004 satellite mass: 5.15855058 Jacobi Radius: 27.92036618643829
snap id: 285
[-65.32 105.74 -46.97] [-50.68 165.34 -39.97]
host mass: 37.451786272 satellite mass: 4.98313436 Jacobi Radius: 25.028903592220928
snap id: 290
[-82.23 108.14 -51.  ] [-49.46 153.34 -43.62]
host mass: 33.15995450200001 satellite mass: 4.7

[ 64.21 209.3   30.19] [ 70.14 106.13  53.2 ]
host mass: 32.77487035 satellite mass: 1.8199489000000004 Jacobi Radius: 32.05825328136608
snap id: 535
[ 71.55 206.01  41.52] [ 72.69 104.51  54.68]
host mass: 32.684617706000004 satellite mass: 1.9186521800000005 Jacobi Radius: 31.573345281053484
snap id: 540
[ 78.93 200.18  53.2 ] [ 75.41 103.26  56.01]
host mass: 32.222933598000004 satellite mass: 1.8766650600000003 Jacobi Radius: 29.850097726435717
snap id: 545
[ 86.31 191.59  64.78] [ 78.07 101.99  57.49]
host mass: 29.174877702 satellite mass: 1.7143456200000005 Jacobi Radius: 27.855480072363314
snap id: 550
[ 93.59 179.89  76.16] [ 80.68 100.44  59.03]
host mass: 29.252494448000004 satellite mass: 1.8090166400000003 Jacobi Radius: 25.829822859835716
snap id: 555
[100.26 165.03  86.96] [83.34 99.14 60.69]
host mass: 27.072503898000004 satellite mass: 1.7591972600000003 Jacobi Radius: 23.269574312594045
snap id: 560
[106.06 146.55  96.33] [85.88 97.81 62.47]
host mass: 24.442444704000

host mass: 20.126702672000004 satellite mass: 1.1056034600000002 Jacobi Radius: 13.017498156149156
snap id: 800
